# Agenda

1. What is data cleaning? Why do we need to clean our data?
2. `NaN`/`NA`
    - What are they? Good and bad
    - What types of values are they, and how does this complicate things (or not)?
    - `dropna`
    - Selective dropping of `NaN`
    - `fillna` to replace values with a scalar
    - `fillna` with a method call
2. Removing bad values
3. `interpolate` -- replacing `NaN` values with reasonable fakes
4. Finding and removing outliers
5. Regularizing string data with strip/lower/replace
6. Finding and removing duplicate data

# What is data cleaning?

When we read in data from somewhere, it will almost always have flaws. That's because:

- human error
- computers going down
- sensors that are wrong (or down)
- people refusing to answer questions

If you read in a data set, it will almost always have problems *or* someone who prepared the data set did the hard work ahead of you.



# NaN / NA

If data is missing, then Pandas usually represents it as `NaN`, short for "not a number." You might have also seen it written as `nan`. They are identical, except that they both used to be defined in NumPy.

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
np.NaN

nan

In [3]:
np.nan is np.NaN

True

In [4]:
type(np.nan)

float

In [5]:
np.nan + 5

nan

In [6]:
np.nan * 100

nan

In [7]:
a = np.array([97, 85, 92, 98, 89])
a.mean()  

92.2

In [8]:
a = np.array([97, 85, np.nan, 98, 89])
a.mean()  

nan

In [9]:
a.dtype

dtype('float64')

In [10]:
a

array([97., 85., nan, 98., 89.])

In [11]:
s = Series([97, 85, 92, 98, 89])
s

0    97
1    85
2    92
3    98
4    89
dtype: int64

In [12]:
s.mean()

92.2

In [13]:
s = Series([97, 85, np.nan, 98, 89])
s

0    97.0
1    85.0
2     NaN
3    98.0
4    89.0
dtype: float64

In [14]:
s.mean()  # by default, Pandas ignores NaN in a series when we perform calculations on it!

92.25

In [16]:
s.mean(skipna=False)  # if you want the NumPy behavior, you can get it like this

nan

In [17]:
np.nan == np.nan  # is NaN equal to itself?

False

In [21]:
# you cannot filter our NaN in this way!
s != np.nan

0    True
1    True
2    True
3    True
4    True
dtype: bool

# NaN vs NA

If I have a series of strings, and one of the values in that series is `NaN`, what will be the dtype of that series? The answer: `object`, but that's true for all string series, because Pandas always uses `object` for strings and anything else it doesn't know what to do with.

What if you have a bunch of integers? You'll turn them into floats.

This is annoying! We want to be able to say that a series contains ints and the occasional `NaN`. But we cannot.

If we try to turn a series of ints + `NaN` into ints, we'll get an error!

In [22]:
s.astype(int)  # get a new series back, where the dtype is int

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [23]:
s

0    97.0
1    85.0
2     NaN
3    98.0
4    89.0
dtype: float64

# Nullable types

Pandas has started to provide us with "nullable types," meaning versions of int/string/etc. that retain their dtype while also allowing us to say that there are missing values. However, these missing values are written as `NA`, or `pd.NA`, which has the same semantics as `NaN` but isn't a float.

In [27]:
s = Series([97, 85, pd.NA, 98, 89], dtype=pd.Int64Dtype())

In [28]:
s

0      97
1      85
2    <NA>
3      98
4      89
dtype: Int64

In [30]:
s.mean()

92.25

In [31]:
s * 5

0     485
1     425
2    <NA>
3     490
4     445
dtype: Int64

In [32]:
s = Series([97, 85, np.nan, 98, 89, np.nan, np.nan, 80])


In [33]:
s

0    97.0
1    85.0
2     NaN
3    98.0
4    89.0
5     NaN
6     NaN
7    80.0
dtype: float64

In [35]:
# how many NaN values are in my series?

# Option 1: use .isna() and run value_counts() on the results
s.isna().value_counts()

False    5
True     3
Name: count, dtype: int64

In [36]:
s.isna().sum()   # because True==1 and False==0

3

In [37]:
# Option 1 for dealing with NaN is just to get rid of it
# the dropna method removes all NaN values in a series

s.dropna()   # this returns a new series (not modifying the original!), containing non-NaN values

0    97.0
1    85.0
3    98.0
4    89.0
7    80.0
dtype: float64

In [39]:
s.fillna(999)  # returns a new series in which all NaN values are replaced by the scalar value we gave

0     97.0
1     85.0
2    999.0
3     98.0
4     89.0
5    999.0
6    999.0
7     80.0
dtype: float64

In [40]:
df = DataFrame([[10, 20, np.nan],
                [40, 50, 60],
                [np.nan, np.nan, 90],
                [100, 110, np.nan]],
               index=list('abcd'),
               columns=list('xyz'))
df

,x,y,z
a,10.0,20.0,NaN
b,40.0,50.0,60.0
c,NaN,NaN,90.0
d,100.0,110.0,NaN


In [41]:
# if I invoke a series method on a data frame, I get back the result
# of invoking that method on each column (which is, after all, a series)

df.mean()

x    50.0
y    60.0
z    75.0
dtype: float64

In [42]:
# what happens if I call dropna on my data frame?
# wherever there is a NaN value, the ENTIRE ROW is dropped

df.dropna()

,x,y,z
b,40.0,50.0,60.0


In [43]:
df.fillna(9999)

,x,y,z
a,10.0,20.0,9999.0
b,40.0,50.0,60.0
c,9999.0,9999.0,90.0
d,100.0,110.0,9999.0


In [46]:
# we can fill differnet columns with different scalar values
df.fillna({'x':999, 'y':888, 'z':777})

,x,y,z
a,10.0,20.0,777.0
b,40.0,50.0,60.0
c,999.0,888.0,90.0
d,100.0,110.0,777.0


In [52]:
df.fillna('')  # this works, but don't do it!

,x,y,z
a,10.0,20.0,
b,40.0,50.0,60.0
c,,,90.0
d,100.0,110.0,


In [47]:
df.mean()   # this returns a series in which the index is df's column names

x    50.0
y    60.0
z    75.0
dtype: float64

In [53]:
df.fillna(df.mean())   # (1) calculate the mean for each column and (2) replace NaN in each column with the column's mean

,x,y,z
a,10.0,20.0,75.0
b,40.0,50.0,60.0
c,50.0,60.0,90.0
d,100.0,110.0,75.0


In [54]:
df

,x,y,z
a,10.0,20.0,NaN
b,40.0,50.0,60.0
c,NaN,NaN,90.0
d,100.0,110.0,NaN


# Exercise: Cleaning data frames from `NaN`

1. Create a simple data frame (like mine), and make sure to have some `NaN` values in there.
2. Calculate the mean for each column.
3. What happens if you use `dropna` on your data frame?
4. What happens if you use `fillna` on your data frame with the result of `df.mean()`?
5. Try the same thing, but using 